In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import PIL
from PIL import Image

In [2]:
'''
长度L1：150~600
长度L2：150~600
宽度W1：20~200
宽度W2：20~200
偏移量offset：-L1/2~L1/2
周期Px：200~800
周期Py：200~800
旋转角度φ：0~180°
'''

'\n长度L1：150~600\n长度L2：150~600\n宽度W1：20~200\n宽度W2：20~200\n偏移量offset：-L1/2~L1/2\n周期Px：200~800\n周期Py：200~800\n旋转角度φ：0~180°\n'

In [3]:
# Set the random seed for reproducibility
random.seed(42)

# Number of data points
num_data_points = 5000000


# Number of segments for each variable
num_segments = 10

# Define the ranges for each parameter
w1_range = (20, 200)
l1_range = (150, 600)
w2_range = (20, 200)
l2_range = (150, 600)
offset_range = (-300, 300)
phi_range = (0, 180)
px_range = (200, 800)
py_range = (200, 800)

# Generate random data
data = {
    'W1': [],
    'L1': [],
    'W2': [],
    'L2': [],
    'Offset': [],
    'Phi': [],
    'Px': [],
    'Py': [],
}


# Generate remaining random data
for _ in range(num_data_points):
    data['W1'].append(random.randint(*w1_range))
    data['L1'].append(random.randint(*l1_range))
    data['W2'].append(random.randint(*w2_range))
    data['L2'].append(random.randint(*l2_range))
    data['Offset'].append(random.randint(*offset_range))
    data['Phi'].append(random.randint(*phi_range))
    data['Px'].append(random.randint(*px_range))
    data['Py'].append(random.randint(*py_range))

# Create a DataFrame
df = pd.DataFrame(data)



In [5]:
# 初步筛选
def filter_data(df):
    """
    过滤数据的函数。

    参数：
    df (DataFrame): 包含原始数据的DataFrame。

    返回：
    DataFrame: 过滤后的DataFrame。
    """
    # 定义过滤条件
    def condition1(df):
        return df['W2'] <= df['Px']

    def condition2(df):
        return df['L1'] <= df['Px']

    def condition3(df):
        return df['W1'] <= df['Py']

    def condition4(df):
        return df['L2'] <= df['Py']

    def condition5(df):
        return df['W2'] <= df['L1']

    def condition6(df):
        return df['W1'] <= df['L2']

    def condition7(df):
        return (abs(df['Offset']) + df["W1"] /2) <= df['L2']/2



    # 使用逻辑与连接所有过滤条件
    combined_condition = pd.Series(True, index=df.index)
    for condition in [condition1,condition2,condition3,condition4,condition5,condition6,condition7]:
        combined_condition = combined_condition & condition(df)

    # 根据过滤条件进行筛选
    filtered_df = df[combined_condition]

    return filtered_df

# 读取数据
# df = pd.read_csv('random_data_with_segments.csv')

# 使用函数过滤数据
filtered_df = filter_data(df)

# 打印过滤前后的数据数量
print(f"过滤前的数据数量: {len(df)}")
print(f"过滤后的数据数量: {len(filtered_df)}")

# 如果需要保存过滤后的数据，可以使用以下代码
# filtered_df.to_csv('filtered_data.csv', index=False)


过滤前的数据数量: 5000000
过滤后的数据数量: 916257


In [6]:
data_50W = filtered_df.sample(n=500000, random_state=42)
data_50W

,W1,L1,W2,L2,Offset,Phi,Px,Py
833606,52,422,85,526,147,88,544,767
4213219,123,279,33,274,-23,14,591,299
4979199,40,171,159,331,-46,61,348,578
4502859,20,388,98,202,-1,159,655,430
1749972,22,405,27,497,139,112,602,714
...,...,...,...,...,...,...,...,...
507150,190,398,176,397,-31,41,737,557
2463822,125,157,52,392,118,85,276,622
3972152,133,395,43,278,-40,160,524,773
831618,194,292,132,524,-94,17,656,722


In [7]:
data_50W.insert(0, 'Pic', None)
data_50W.insert(0, 'type', "double_rec")

In [8]:
# 将索引重置为从0开始的连续整数
data_50W.reset_index(drop=True, inplace=True)
data_50W

,type,Pic,W1,L1,W2,L2,Offset,Phi,Px,Py
0,double_rec,None,52,422,85,526,147,88,544,767
1,double_rec,None,123,279,33,274,-23,14,591,299
2,double_rec,None,40,171,159,331,-46,61,348,578
3,double_rec,None,20,388,98,202,-1,159,655,430
4,double_rec,None,22,405,27,497,139,112,602,714
...,...,...,...,...,...,...,...,...,...,...
499995,double_rec,None,190,398,176,397,-31,41,737,557
499996,double_rec,None,125,157,52,392,118,85,276,622
499997,double_rec,None,133,395,43,278,-40,160,524,773
499998,double_rec,None,194,292,132,524,-94,17,656,722


In [10]:
from tqdm import tqdm
for i in tqdm(range(len(data_50W))):
    W1 = data_50W.iloc[i]['W1']
    L1 = data_50W.iloc[i]['L1']
    W2 = data_50W.iloc[i]['W2']
    L2 = data_50W.iloc[i]['L2']
    offset = data_50W.iloc[i]['Offset']
    phi = data_50W.iloc[i]['Phi']
    Px = data_50W.iloc[i]['Px']
    Py = data_50W.iloc[i]['Py']
    pic_name = f'W1={W1}=L1={L1}=W2={W2}=L2={L2}=offset={offset}=phi={phi}=Px={Px}=Py={Py}.png'
    data_50W.loc[i,"Pic"] = pic_name

100%|██████████| 500000/500000 [03:45<00:00, 2221.72it/s]


In [12]:
data_50W.to_csv('cross_50w.csv', index=False)

In [2]:
data = pd.read_csv('cross_50w.csv')

In [3]:
data.insert(1,"filter",None)
data
# True:需要筛选 False：符合要求不需要筛选

,type,filter,Pic,W1,L1,W2,L2,Offset,Phi,Px,Py
0,double_rec,None,W1=52=L1=422=W2=85=L2=526=offset=147=phi=88=Px...,52,422,85,526,147,88,544,767
1,double_rec,None,W1=123=L1=279=W2=33=L2=274=offset=-23=phi=14=P...,123,279,33,274,-23,14,591,299
2,double_rec,None,W1=40=L1=171=W2=159=L2=331=offset=-46=phi=61=P...,40,171,159,331,-46,61,348,578
3,double_rec,None,W1=20=L1=388=W2=98=L2=202=offset=-1=phi=159=Px...,20,388,98,202,-1,159,655,430
4,double_rec,None,W1=22=L1=405=W2=27=L2=497=offset=139=phi=112=P...,22,405,27,497,139,112,602,714
...,...,...,...,...,...,...,...,...,...,...,...
499995,double_rec,None,W1=190=L1=398=W2=176=L2=397=offset=-31=phi=41=...,190,398,176,397,-31,41,737,557
499996,double_rec,None,W1=125=L1=157=W2=52=L2=392=offset=118=phi=85=P...,125,157,52,392,118,85,276,622
499997,double_rec,None,W1=133=L1=395=W2=43=L2=278=offset=-40=phi=160=...,133,395,43,278,-40,160,524,773
499998,double_rec,None,W1=194=L1=292=W2=132=L2=524=offset=-94=phi=17=...,194,292,132,524,-94,17,656,722


In [4]:
from PIL import Image
# Image.open(os.path.join(droplist["dir"][1520],droplist["name"][1520]+".png"))

import cv2
import numpy as np
from tqdm import tqdm

def check_check(img,Px,Py):

    # 读取图像
    image = cv2.imread(img)

    # 计算图像中心坐标
    image_center_x = image.shape[1] // 2
    image_center_y = image.shape[0] // 2


    # 定义矩形框尺寸
    rect_width, rect_height = Px//9, Py//9

    # 计算矩形框左上角坐标
    rect_x = image_center_x - rect_width // 2
    rect_y = image_center_y - rect_height // 2

    # 定义红色范围
    red = np.array([0, 0, 255])

    # 检查框外的红色像素
    is_red_outside = False
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            # 检查是否在框外

            if not (rect_x <= x < rect_x + rect_width and rect_y <= y < rect_y + rect_height):
                # 检查像素是否为红色
                if np.array_equal(image[y, x], red):
                    is_red_outside = True
                    break
        if is_red_outside:
            break

    # 输出结果
    if is_red_outside:
        # print('检测到红色像素在矩形框外')
        return True
    else:
        # print('未检测到红色像素在矩形框外')
        return False

    # 绘制图像和矩形框（可选）
    # cv2.rectangle(image, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), (0, 255, 0), 2)
    # cv2.imshow('Image with Rectangle', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [5]:
dirr = "D:\simulation\BMD\cross\pic_expand_prepare"
for i in tqdm(range(len(data))):
    name = data["Pic"][i]
    Px = data.iloc[i]['Px']
    Py = data.iloc[i]['Py']
    data.loc[i,"filter"] = check_check(os.path.join(dirr,name),int(Px),int(Py))

100%|██████████| 500000/500000 [3:21:44<00:00, 41.31it/s]  


In [7]:
data_part_1 = data[data["filter"]==False]
data_part_1

,type,filter,Pic,W1,L1,W2,L2,Offset,Phi,Px,Py
0,double_rec,False,W1=52=L1=422=W2=85=L2=526=offset=147=phi=88=Px...,52,422,85,526,147,88,544,767
1,double_rec,False,W1=123=L1=279=W2=33=L2=274=offset=-23=phi=14=P...,123,279,33,274,-23,14,591,299
3,double_rec,False,W1=20=L1=388=W2=98=L2=202=offset=-1=phi=159=Px...,20,388,98,202,-1,159,655,430
4,double_rec,False,W1=22=L1=405=W2=27=L2=497=offset=139=phi=112=P...,22,405,27,497,139,112,602,714
5,double_rec,False,W1=164=L1=446=W2=28=L2=333=offset=22=phi=75=Px...,164,446,28,333,22,75,743,540
...,...,...,...,...,...,...,...,...,...,...,...
499993,double_rec,False,W1=148=L1=187=W2=166=L2=374=offset=63=phi=9=Px...,148,187,166,374,63,9,778,434
499994,double_rec,False,W1=28=L1=468=W2=53=L2=476=offset=213=phi=92=Px...,28,468,53,476,213,92,795,735
499995,double_rec,False,W1=190=L1=398=W2=176=L2=397=offset=-31=phi=41=...,190,398,176,397,-31,41,737,557
499997,double_rec,False,W1=133=L1=395=W2=43=L2=278=offset=-40=phi=160=...,133,395,43,278,-40,160,524,773


In [8]:
data_part_1.to_csv("D:\simulation\BMD\cross_filter.csv")

In [9]:
data_part_1.sample(n=350000).to_csv("D:\simulation\BMD\cross_35w.csv")